# Predictions Overview

This is a small notebook to report and analyse Prediction Studio data on Predictions. The underlying data is from the Data-DM-Snapshot table that is used to populate the Prediction Studio screen with Prediction Performance, Lift, CTR etc.

As data this notebook accept data exported from PDC - which has a slightly altered format - as well as data directly exported from the pyGetSnapshot dataset in Pega.

For a description of the datamart tables see https://docs-previous.pega.com/decision-management/87/database-tables-monitoring-models.

Disclaimer: this is not a canned, robust and customer-facing notebook (yet). It's mostly used internally to validate Prediction data. Column names and file formats may need some more review to make it more robust.

## Raw data

First, we're going to show the raw data. The raw data is in a "long" format with e.g. test and control groups in separate rows.

In [1]:
from pathlib import Path
import sys
import polars as pl
import json
from pdstools import read_ds_export, Prediction

# path to dataset export here 
# e.g. PR_DATA_DM_SNAPSHOTS.parquet
data_export = "<Your Export Here>" 

prediction = None
if data_export.endswith(".parquet"):
    predictions_raw_data = pl.read_parquet(Path(data_export).expanduser())
    prediction = Prediction(predictions_raw_data.lazy())
elif data_export.endswith(".json"):
    print("Import of PDC JSON data not supported")
    sys.exit()
elif data_export.endswith(".zip"):
    # Assuming a direct export from the dataset
    predictions_raw_data = read_ds_export(data_export).collect()
    prediction = Prediction(predictions_raw_data.lazy())

predictions_raw_data.head().to_pandas().style

NameError: name 'predictions_raw_data' is not defined

## Prediction Data

The actual prediction data is in a "wide" format with separate fields for Test and Control groups. Also, it is only the "daily" snapshots and the numbers and date are formatted to be normal Polars types.

In [ ]:
prediction.predictions.head().collect().to_pandas().style

## Summary by Channel

Standard functionality exists to summarize the predictions per channel. Note that we do not have the prediction to channel mapping in the data (this is an outstanding product issue), so apply the implicit naming conventions of NBAD. For a specific customer, custom mappings can be passed into the summarization function.

In [ ]:
prediction.summary_by_channel().collect().to_pandas().style

# Prediction Trends

Summarization by default is over all time. You can pass in an argument to summarize by day, week or any other period as supported by the (Polars time offset string language)[https://docs.pola.rs/api/python/stable/reference/expressions/api/polars.Expr.dt.offset_by.html].

This trend data can then easily be visualized.

In [ ]:
prediction.plot.performance_trend("1w")

In [ ]:
prediction.plot.lift_trend("1w")

In [ ]:


prediction.plot.ctr_trend("1w", facetting=True)


In [ ]:


prediction.plot.responsecount_trend("1w", facetting=False)